In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date
import time
today = date.today()
path = os.getcwd()
print(f'💚 Today is {today}')

💚 Today is 2025-06-05


# Neo4j Implementation of the Graph API

In [2]:
# Load your CSV
df = pd.read_csv(f'{path}/data_abstracts/true_mobility_studies_617_forKGs.csv')
df.columns

Index(['id', 'title', 'year', 'doi', 'landing_page', 'abstract_inverted_index',
       'language', 'is_oa', 'oa_status', 'oa_link', 'abstract', 'abstract_sm',
       'authors_sm', 'domain', 'sort_gpt_1', 'sort_gpt_2', 'sort_gpt4o_1',
       'sort_gpt4o_2', 'cited_by', 'len_cited_by', 'ref_count',
       'cited_by_count', 'Q1', 'Q1_1', 'Q2', 'Q2_1', 'Q2_2', 'Q3', 'Q4', 'abs',
       'index', 'category_n1', 'measure', 'specified', 'relevant',
       'category_n2', 'measure_1', 'measure_2', 'category_3', 'file', 'id.1',
       'Question1', 'Question2', 'rq_cat', 'RQ', 'data_cat', 'data_type',
       'type', 'category_1', 'category_2', 'author_raw_names',
       'author_raw_affiliations', 'topic_display_names', 'topic_scores',
       'field_display_names', 'subfield_display_names'],
      dtype='object')

In [3]:
df[df['data_cat']==0][['data_cat', 'data_type']].values # This shouldn't be 0!

array([[0,
        'European Union Statistics on Income and Living Conditions survey: 3; European Community Household Panel: 7']],
      dtype=object)

In [4]:
df['data_cat'] = df['data_cat'].replace({0: 3})
df[df['data_cat'] == 0][['data_cat', 'data_type']].values

array([], shape=(0, 2), dtype=object)

In [5]:
new_rq = {1: 'Measurement and Methodological Advances', 2: 'Empirical Estimates and Determinants', 
          3: 'Policy, Institutional, and Geographic Impacts', 4: 'Intergenerational Wealth Mobility and Inheritance', 
          5: 'Demographic Differences in Mobility (Race, Gender, etc.)', 6: 'Mobility and Non-Income Outcomes (Health, Wellbeing, etc.)', 
          7: 'Theoretical and Structural Models', 8: 'Perceptions of Mobility and Attitudes', 9: 'Others'}


new_data = {1: 'Panel/Longitudinal Surveys', 2: 'Administrative/Registry Data',
            3: 'National Survey Data', 4: 'Opportunity Atlas',
            5: 'Natural/Experimental Data', 6: 'Linked Administrative Data',
            7: 'International Panel Data', 8: 'Rich List Data',
            9: 'University/Institution Data', 10: 'Pseudo-Panel/Household Budget Survey',
            11: 'Archival/Historical Data', 12: 'Big Data',
            13: 'No dataset', 14: 'Others'}

df['rq_cat'] = df['rq_cat'].replace(new_rq)
df['data_cat'] = df['data_cat'].replace(new_data)
df['rq_cat'] = df['rq_cat'].astype('category')
df['data_cat'] = df['data_cat'].astype('category')

df[['rq_cat', 'data_cat']].head(5)

,rq_cat,data_cat
0,Intergenerational Wealth Mobility and Inheritance,Linked Administrative Data
1,Empirical Estimates and Determinants,No dataset
2,Empirical Estimates and Determinants,No dataset
3,Measurement and Methodological Advances,Panel/Longitudinal Surveys
4,Empirical Estimates and Determinants,Others


In [6]:
df['type'] = df['type'].replace({None: 'Not_specified'})

In [7]:
df[['year', 'rq_cat', 'data_cat', 'type', 'category_1', 'category_2']].isna().sum()

year            0
rq_cat          0
data_cat        0
type            0
category_1      0
category_2    371
dtype: int64

In [19]:
# df.to_csv(f'{path}/data_abstracts/true_mobility_studies_617_forKGs_cleaned.csv', index=False)

In [20]:
df[['id','cited_by']].head()

,id,cited_by
0,https://openalex.org/W2564827629,"https://openalex.org/W2763922368, https://open..."
1,https://openalex.org/W2117407660,"https://openalex.org/W1966248378, https://open..."
2,https://openalex.org/W1515121029,"https://openalex.org/W1984539215, https://open..."
3,https://openalex.org/W1966505070,"https://openalex.org/W1577807578, https://open..."
4,https://openalex.org/W3041900243,"https://openalex.org/W4386370708, https://open..."


In [8]:
df_citations = df.dropna(subset=['cited_by'])
df_citations = df_citations[['id', 'cited_by']]
df_citations['cited_by'] = df_citations['cited_by'].apply(lambda x: [s.strip() for s in x.split(',')])
df_citations = df_citations.explode('cited_by')
df_citations = df_citations.rename(columns={'cited_by': 'source_id', "id": "target_id"})

valid_ids = set(df['id'])
df_citations = df_citations[df_citations['source_id'].isin(valid_ids)]

print(f'\nNumber of citations: {len(df_citations)}')
df_citations.head(10)



Number of citations: 2596


,target_id,source_id
0,https://openalex.org/W2564827629,https://openalex.org/W3123816425
0,https://openalex.org/W2564827629,https://openalex.org/W4289222991
0,https://openalex.org/W2564827629,https://openalex.org/W4386370708
0,https://openalex.org/W2564827629,https://openalex.org/W3163153651
0,https://openalex.org/W2564827629,https://openalex.org/W4393244952
0,https://openalex.org/W2564827629,https://openalex.org/W4407152405
0,https://openalex.org/W2564827629,https://openalex.org/W2947666646
0,https://openalex.org/W2564827629,https://openalex.org/W3022704615
0,https://openalex.org/W2564827629,https://openalex.org/W4280580662
0,https://openalex.org/W2564827629,https://openalex.org/W4283657131


In [9]:
df['rq_cat'].unique().tolist()

['Intergenerational Wealth Mobility and Inheritance',
 'Empirical Estimates and Determinants',
 'Measurement and Methodological Advances',
 'Theoretical and Structural Models',
 'Policy, Institutional, and Geographic Impacts',
 'Mobility and Non-Income Outcomes (Health, Wellbeing, etc.)',
 'Perceptions of Mobility and Attitudes',
 'Demographic Differences in Mobility (Race, Gender, etc.)',
 'Others']

In [10]:
import pandas as pd
from neo4j import GraphDatabase

# Connect to Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "your_password"))
df['category_1'] = df['category_1'].replace({'Others': 'Others_Measure'})
df['category_2'] = df['category_2'].replace({'Others': 'Others_Measure'})
df['data_cat'] = df['data_cat'].replace({'Others': 'Others_DataType'})
df['rq_cat'] = df['rq_cat'].replace({'Others': 'Others_RqType'})


with driver.session() as session:
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (y:Year) REQUIRE y.name IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Measure) REQUIRE m.name IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (ds:DataType) REQUIRE ds.name IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (r:RqType) REQUIRE r.name IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (tp:Type) REQUIRE tp.name IS UNIQUE")

def safe_str(val):
    """Return string if not null/NaN, else None."""
    if pd.isna(val) or str(val).strip().lower() in {"", "nan", "none"}:
        return None
    return str(val).strip()

def create_graph(tx, row):
    # Sanitize all inputs
    article_id = safe_str(row.get("id"))
    year = int(row["year"]) if not pd.isna(row["year"]) else None
    measure1 = safe_str(row.get("category_1"))
    measure2 = safe_str(row.get("category_2"))
    datatype = safe_str(row.get("data_cat"))
    rqtype = safe_str(row.get("rq_cat"))
    type = safe_str(row.get("type"))

    # Skip the row if any required values are None
    if not all([article_id, year, datatype, rqtype, type]):
        print(f"[SKIP] Missing required data in row: {row.get('id')}")
        return  # skip this row

    # Skip measure1 or measure2 if they're None, and adjust the Cypher
    query = """
    MERGE (p:Paper {id: $article_id})
    MERGE (y:Year {name: $year})
    MERGE (m1:Measure {name: $measure1})
    MERGE (ds:DataType {name: $datatype})
    MERGE (rq:RqType {name: $rqtype}) 
    MERGE (tp:Type {name: $type})

    // Supernodes 
    MERGE (p)-[:HAS_YEAR]->(y)
    MERGE (p)-[:HAS_TYPE]->(tp)
    MERGE (p)-[:HAS_DATATYPE]->(ds)
    MERGE (p)-[:HAS_RQTYPE]->(rq)
    MERGE (p)-[:HAS_MEASURE]->(m1)

    // Properties
    MERGE (y)-[:PERFORMS {article_id: $article_id}]->(tp)
    MERGE (tp)-[:USES {article_id: $article_id}]->(m1)
    MERGE (m1)-[:APPLIED_WITH {article_id: $article_id}]->(ds)
    MERGE (ds)-[:TO_ANSWER {article_id: $article_id}]->(rq)
    MERGE (rq)-[:IS_ANSWERED_IN {article_id: $article_id}]->(y)
    """
    
    if measure2:
        query += "\nMERGE (m2:Measure {name: $measure2})\nMERGE (p)-[:HAS_MEASURE]->(m2)\nMERGE (tp)-[:USES {article_id: $article_id}]->(m2)\nMERGE (m2)-[:APPLIED_WITH {article_id: $article_id}]->(ds)"

    tx.run(query, {
        "year": year,
        "measure1": measure1,
        "measure2": measure2,
        "datatype": datatype,
        "rqtype": rqtype,
        "type": type,
        "article_id": article_id,
    })
    

# Loop through DataFrame and create graph
with driver.session() as session:
    for _, row in df.iterrows():
        session.execute_write(create_graph, row)



# Create the citation relationships
with driver.session() as session:
    query = """
    UNWIND $rows AS row
    MATCH (a:Paper {id: row.source_id})
    MATCH (b:Paper {id: row.target_id})
    MERGE (a)-[:CITES]->(b)
    """
    session.run(query, parameters={"rows": df_citations.to_dict("records")})

driver.close()


/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_54808/937208623.py:8: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['data_cat'] = df['data_cat'].replace({'Others': 'Others_DataType'})
/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_54808/937208623.py:9: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['rq_cat'] = df['rq_cat'].replace({'Others': 'Others_RqType'})


# Analysis

In [88]:
ids_to_check = ["https://openalex.org/W4308132439", "https://openalex.org/W4285263003", "https://openalex.org/W4286458987", "https://openalex.org/W4296277673", "https://openalex.org/W4304172618"]
df[df['id'].isin(ids_to_check)][['measure', 'RQ', 'data_type']].values

array([['Regression-based measures (regression analysis of wage returns linking current outcomes with past occupational and intergenerational data)',
        'How does return migration influence intertemporal (and potentially intergenerational) socioeconomic mobility and labor market outcomes (e.g., wage returns) among Jordanian male workers, and what are the key socioeconomic and migration-related determinants driving these outcomes?',
        'Jordanian Labor Market Panel Survey'],
       ['The paper estimates intergenerational income mobility using regression‐based measures (likely a form of intergenerational elasticity, even if not explicitly labeled as IGE)',
        'How does trade opening—specifically through changes in the proportion of trade exports and export structure—affect intergenerational income mobility? And through what mechanisms, such as improvements in education, do these trade variables influence the upward mobility of income, particularly among urban residents?',


In [91]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "your_password"))

def project_and_run_fastrp():
    with driver.session() as session:
        # Drop old graph
        session.run("CALL gds.graph.drop('feature_graph', false) YIELD graphName")

        # Project without article_id
        projection_query = """
        CALL gds.graph.project(
          'feature_graph',
          ['Year', 'Type', 'Measure', 'DataType', 'RqType'],
          ['PERFORMS', 'USES', 'APPLIED_WITH', 'TO_ANSWER', 'IS_ANSWERED_IN']
        )
        """
        session.run(projection_query)

        # Run FastRP
        fastrp_query = """
        CALL gds.fastRP.write(
          'feature_graph',
          {
            embeddingDimension: 128,
            writeProperty: 'embedding'
          }
        ) YIELD nodePropertiesWritten
        """
        session.run(fastrp_query)

project_and_run_fastrp()

In [ ]:
def get_article_embeddings():
    with driver.session() as session:
        query = """
            MATCH (n)
            WHERE n.embedding IS NOT NULL
            UNWIND [(n)-[r]->() WHERE r.article_id IS NOT NULL | r.article_id] AS article_id
            RETURN id(n) AS node_id, article_id, n.embedding AS embedding
        """
        result = session.run(query)
        records = [(r["article_id"], r["embedding"]) for r in result]

        # Convert to DataFrame
        df = pd.DataFrame(records, columns=["article_id", "embedding"])
        df["embedding"] = df["embedding"].apply(np.array)

        # Average embeddings per article_id
        df_avg = (
            df.groupby("article_id")["embedding"]
            .apply(lambda vectors: np.mean(np.vstack(vectors), axis=0)) #Flatten the list of arrays
            .reset_index()
        )

        df_avg.columns = ["article_id", "embedding"] # Embedding would have 128 values in it
        return df_avg  # ✅ This is the key fix

# Run it
df_embeddings = get_article_embeddings()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 20, offset: 177} for query: '\n            MATCH (n)\n            WHERE n.embedding IS NOT NULL\n            UNWIND [(n)-[r]->() WHERE r.article_id IS NOT NULL | r.article_id] AS article_id\n            RETURN id(n) AS node_id, article_id, n.embedding AS embedding\n        '


In [105]:
df_embeddings.head(5)

,article_id,embedding
0,https://openalex.org/W1020822704,"[-0.11972517669200897, -0.10026626214385033, 0..."
1,https://openalex.org/W115198923,"[-0.12046551704406738, -0.08210444301366807, 0..."
2,https://openalex.org/W1444704346,"[-0.11972517669200897, -0.10026626214385033, 0..."
3,https://openalex.org/W146920992,"[-0.11447846357311521, -0.061220607055085044, ..."
4,https://openalex.org/W1502476793,"[-0.11886386573314667, -0.10489165186882018, 0..."


In [116]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

X = np.vstack(df_embeddings["embedding"].values)
kmeans = KMeans(n_clusters=5, random_state=42).fit(X)
df_embeddings["cluster"] = kmeans.labels_

# For 2D visualization
pca = PCA(n_components=2)
coords = pca.fit_transform(X)

In [131]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA

df["cluster_code"] = df["cluster"].astype("category").cat.codes
df["field_code"] = df["field_display_names"].astype("category").cat.codes
df["category_code"] = df["category_1"].astype("category").cat.codes
df["datatype_code"] = df["data_cat"].astype("category").cat.codes
df["rq_code"] = df["rq_cat"].astype("category").cat.codes
df["type_code"] = df["type"].astype("category").cat.codes
df["oa_code"] = df["is_oa"].astype("category").cat.codes
df["year_code"] = df["year"].astype("category").cat.codes
df["topic_code"] = df["topic_display_names"].astype("category").cat.codes
df["subfield_code"] = df["subfield_display_names"].astype("category").cat.codes
df["affiliation_code"] = df["author_raw_affiliations"].astype("category").cat.codes
df["citations_code"] = df["cited_by_count"].astype("category").cat.codes


trace = go.Scatter(
    x=coords[:, 0],
    y=coords[:, 1],
    mode='markers',
    marker=dict(
        color=df["cluster_code"],
        colorscale='Viridis',
        colorbar=dict(title="Cluster"),
        size=5,
    ),
    text=df["id"],  # Optional: show article_id on hover
    hoverinfo="text",
    showlegend=False,
)

# 4. Centroid approximation in 2D PCA space
centroids_umap = np.array([
    coords[df["cluster_code"] == i].mean(axis=0)
    for i in sorted(df["cluster_code"].unique())
])

centroid_trace = go.Scatter(
    x=centroids_umap[:, 0],
    y=centroids_umap[:, 1],
    mode='markers',
    name='Centroids',
    marker=dict(
        color='red',
        size=10,
        symbol='x'
    ),
    text=[f'Cluster {i}' for i in sorted(df["cluster_code"].unique())],
    hoverinfo='text',
    showlegend=False,
)

# 5. Dropdown layout
layout = go.Layout(
    title='KMeans Visualization of Papers',
    paper_bgcolor='white',
    updatemenus=[dict(
        type='dropdown',
        showactive=True,
        x=0.17,
        xanchor='left',
        y=1.15,
        yanchor='top',
        buttons=[
            {'label': 'Cluster', 'method': 'update', 'args': [{'marker.color': [df['cluster_code']]}]},
            {'label': 'Field', 'method': 'update', 'args': [{'marker.color': [df['field_code']]}]},
            {'label': 'Category', 'method': 'update', 'args': [{'marker.color': [df['category_code']]}]},
            {'label': 'Data Category', 'method': 'update', 'args': [{'marker.color': [df['datatype_code']]}]},
            {'label': 'RQ Category', 'method': 'update', 'args': [{'marker.color': [df['rq_code']]}]},
            {'label': 'Type', 'method': 'update', 'args': [{'marker.color': [df['type_code']]}]},
            {'label': 'Open Access', 'method': 'update', 'args': [{'marker.color': [df['oa_code']]}]},
            {'label': 'Year', 'method': 'update', 'args': [{'marker.color': [df['year_code']]}]},
            {'label': 'Topic', 'method': 'update', 'args': [{'marker.color': [df['topic_code']]}]},
            {'label': 'Subfield', 'method': 'update', 'args': [{'marker.color': [df['subfield_code']]}]},
            {'label': 'Affiliation', 'method': 'update', 'args': [{'marker.color': [df['affiliation_code']]}]},
            {'label': 'Cited_by_Count', 'method': 'update', 'args': [{'marker.color': [df['citations_code']]}]},
        ]
    )]
)

# 6. Final plot
fig = go.Figure(data=[trace, centroid_trace], layout=layout)
fig.show()


In [ ]:
# df_embeddings.to_csv(f'{path}/data_abstracts/subgraphs_617_fastRP_embeddings.csv', index=False)

# Graph Structure Analysis

* After adding the year nodes, it looks more concentrated.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import math

# Load your CSV
df_feature_edges = pd.read_csv(f'{path}/results/weights_properties_only_noTypeYear.csv')

# Build the graph
G = nx.DiGraph()
for _, row in df_feature_edges.iterrows():
    G.add_edge(row['from_val'], row['to_val'], weight=row['weight'])

# Layout
pos = nx.spring_layout(G, seed=42, k=5 / math.sqrt(G.order()))

# Edge thickness based on weight
edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
min_w, max_w = min(edge_weights), max(edge_weights)

def scale_weight(w, min_w, max_w, min_thick=1, max_thick=8):
    if max_w == min_w:
        return (min_thick + max_thick) / 2
    return min_thick + (w - min_w) / (max_w - min_w) * (max_thick - min_thick)

scaled_widths = [scale_weight(w, min_w, max_w) for w in edge_weights]

edge_traces = []
for (u, v), width in zip(G.edges(), scaled_widths):
    edge_traces.append(go.Scatter(
        x=[pos[u][0], pos[v][0], None],
        y=[pos[u][1], pos[v][1], None],
        mode='lines',
        line=dict(width=width, color='gray'),
        showlegend=False,
        hoverinfo='text',
        text=[f'{u} → {v}<br>Weight: {G[u][v]["weight"]}']
    ))

# Discrete color palette (31 colors)
# discrete_colors_31 = [
#     , "#F4A988", "#CB7729", "#C04E3F", "#F7A7A1",
#     "#339999", "#005F56", "#F4E1A1", "#998250", "#A6D785", "#FFD4B2",
#     "#B36200", "#9B2F2F", "#FEC5C1", "#5FBFC0", "#004B47", "#F9EAB0",
#     "#B3A369", "#6DA47B", "#D4816E", "#A35C18", "#932A25", "#D7827F",
#     "#227C7C", "#003B36", "#E6D9A3", "#7E6E3F", "#CCCCCC", "#999999",
#     "#666666", "#333333"
# ]

# Map node labels to color
node_labels = list(G.nodes())
x_pos = [pos[node][0] for node in node_labels]
y_pos = [pos[node][1] for node in node_labels]
node_to_color = {label: i for i, label in enumerate(sorted(node_labels))}
node_colors = [discrete_colors_31[node_to_color[label] % len(discrete_colors_31)] for label in node_labels]

# Create individual node traces with legends
node_traces = []
for i, label in enumerate(node_labels):
    node_traces.append(go.Scatter(
        x=[x_pos[i]],
        y=[y_pos[i]],
        mode='markers',
        name=label,
        marker=dict(
            size=20,
            color=node_colors[i],
            line=dict(width=2, color='darkblue')
        ),
        hoverinfo='text',
        text=[label],
        showlegend=True
    ))

# Final plot
fig = go.Figure(data=edge_traces + node_traces, layout=go.Layout(
    title='Feature Citation Graph with Discrete Colors and Legend',
    showlegend=True,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False)
))

fig.show()


### Zoom into the center?

In [35]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import math

# Load your CSV
df = pd.read_csv(f'{path}/results/weights_properties_only_noTypeYear.csv')
zoom_in_values = ['Regression-based Measures','Rank‐based Measures', 'Transition Matrix / Probability Measures', 'Absolute Mobility Measures', 
                  'Others_Measure', 'Decomposition / Structural Approaches',
                  'Panel/Longitudinal Surveys', 'Administrative/Registry Data', 'National Survey Data', 'Linked Administrative Data',
                  'International Panel Data', 'Others_DataType']
df_feature_edges = df[df['from_val'].isin(zoom_in_values) & df['to_val'].isin(zoom_in_values)]

# Build the graph
G = nx.DiGraph()
for _, row in df_feature_edges.iterrows():
    G.add_edge(row['from_val'], row['to_val'], weight=row['weight'])

# Layout
pos = nx.spring_layout(G, seed=42, k=5 / math.sqrt(G.order()))

# Edge thickness based on weight
edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
min_w, max_w = min(edge_weights), max(edge_weights)

def scale_weight(w, min_w, max_w, min_thick=1, max_thick=8):
    if max_w == min_w:
        return (min_thick + max_thick) / 2
    return min_thick + (w - min_w) / (max_w - min_w) * (max_thick - min_thick)

scaled_widths = [scale_weight(w, min_w, max_w) for w in edge_weights]

edge_traces = []
for (u, v), width in zip(G.edges(), scaled_widths):
    edge_traces.append(go.Scatter(
        x=[pos[u][0], pos[v][0], None],
        y=[pos[u][1], pos[v][1], None],
        mode='lines',
        line=dict(width=width, color='gray'),
        showlegend=False,
        hoverinfo='text',
        text=[f'{u} → {v}<br>Weight: {G[u][v]["weight"]}']
    ))

# Discrete color palette (31 colors)
discrete_colors_31 = [
    "#B5DBA5", "#F4A988", "#CB7729", "#C04E3F", "#F7A7A1",
    "#339999", "#005F56", "#F4E1A1", "#998250", "#A6D785", "#FFD4B2",
    "#B36200", "#9B2F2F", "#FEC5C1", "#5FBFC0", "#004B47", "#F9EAB0",
    "#B3A369", "#6DA47B", "#D4816E", "#A35C18", "#932A25", "#D7827F",
    "#227C7C", "#003B36", "#E6D9A3", "#7E6E3F", "#CCCCCC", "#999999",
    "#666666", "#333333"
]

# Map node labels to color
node_labels = list(G.nodes())
x_pos = [pos[node][0] for node in node_labels]
y_pos = [pos[node][1] for node in node_labels]
node_to_color = {label: i for i, label in enumerate(sorted(node_labels))}
node_colors = [discrete_colors_31[node_to_color[label] % len(discrete_colors_31)] for label in node_labels]

# Create individual node traces with legends
node_traces = []
for i, label in enumerate(node_labels):
    node_traces.append(go.Scatter(
        x=[x_pos[i]],
        y=[y_pos[i]],
        mode='markers',
        name=label,
        marker=dict(
            size=20,
            color=node_colors[i],
            line=dict(width=2, color='darkblue')
        ),
        hoverinfo='text',
        text=[label],
        showlegend=True
    ))

# Final plot
fig = go.Figure(data=edge_traces + node_traces, layout=go.Layout(
    showlegend=True,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False)
))

fig.show()


Measures vs. Data types construction haha ^^;
⚠️ So actually, not considering the citation it not interesting at all because then the weights are just how many times certain nodes are mentioned, and the relationships between the nodes are just the "node types" relationships.

### Citations considered to create weights

In [43]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import math

# Load your CSV
df_feature_edges = pd.read_csv(f'{path}/results/weights_properties_citations_noTypeYear.csv')

# Build the graph
G = nx.DiGraph()
for _, row in df_feature_edges.iterrows():
    G.add_edge(row['from_val'], row['to_val'], weight=row['weight'])

# Layout
pos = nx.spring_layout(G, seed=42, k=5 / math.sqrt(G.order()))

# Edge thickness based on weight
edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
min_w, max_w = min(edge_weights), max(edge_weights)

def scale_weight(w, min_w, max_w, min_thick=1, max_thick=8):
    if max_w == min_w:
        return (min_thick + max_thick) / 2
    return min_thick + (w - min_w) / (max_w - min_w) * (max_thick - min_thick)

scaled_widths = [scale_weight(w, min_w, max_w) for w in edge_weights]

edge_traces = []
for (u, v), width in zip(G.edges(), scaled_widths):
    edge_traces.append(go.Scatter(
        x=[pos[u][0], pos[v][0], None],
        y=[pos[u][1], pos[v][1], None],
        mode='lines',
        line=dict(width=width, color='gray'),
        showlegend=False,
        hoverinfo='text',
        text=[f'{u} → {v}<br>Weight: {G[u][v]["weight"]}']
    ))


color_labels = {
                # Measure Categories
                "Regression‐based Measures" : "#B5DBA5",
                "Rank‐based Measures": "#B5DBA5",
                "Transition Matrix / Probability Measures": "#B5DBA5",
                "Absolute Mobility Measures": "#B5DBA5",
                "Multigenerational Measures": "#B5DBA5",
                "Decomposition / Structural Approaches": "#B5DBA5",
                "Non‐parametric Approaches": "#B5DBA5",
                "Others_Measure": "#B5DBA5",
                
                # Data Categories
                "Panel/Longitudinal Surveys": "#339999",
                "Administrative/Registry Data": "#339999",
                "National Survey Data": "#339999",
                "Opportunity Atlas": "#339999",
                "Natural/Experimental Data": "#339999",
                "Linked Administrative Data": "#339999",
                "International Panel Data": "#339999",
                "Rich List Data": "#339999",
                "University/Institution Data": "#339999",
                "Pseudo-Panel/Household Budget Survey": "#339999",
                "Archival/Historical Data": "#339999",
                "Big Data": "#339999",
                "No dataset": "#339999",
                "Others_DataType": "#339999",
                
                # RQ Categories
                "Measurement and Methodological Advances": "#F4A988",
                "Empirical Estimates and Determinants": "#F4A988",
                "Policy, Institutional, and Geographic Impacts": "#F4A988",
                "Intergenerational Wealth Mobility and Inheritance": "#F4A988",
                "Demographic Differences in Mobility (Race, Gender, etc.)" : "#F4A988",
                "Mobility and Non-Income Outcomes (Health, Wellbeing, etc.)": "#F4A988",
                "Theoretical and Structural Models": "#F4A988",
                "Perceptions of Mobility and Attitudes": "#F4A988",
                "Others_RqType": "#F4A988"}


# Map node labels to color
node_labels = list(G.nodes())
x_pos = [pos[node][0] for node in node_labels]
y_pos = [pos[node][1] for node in node_labels]

# Create individual node traces with legends
node_traces = []
for i, label in enumerate(node_labels):
    node_traces.append(go.Scatter(
        x=[x_pos[i]],
        y=[y_pos[i]],
        mode='markers',
        name=label,
        marker=dict(
            size=20,
            color=color_labels[label],
            line=dict(width=2, color='darkblue')
        ),
        hoverinfo='text',
        text=[label],
        showlegend=True
    ))

# Final plot
fig = go.Figure(data=edge_traces + node_traces, layout=go.Layout(
    showlegend=True,
    hovermode='closest',
    # paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(b=20, l=5, r=5, t=40),
    font=dict(size=16, color='black'),
    font_family="Times New Roman",
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False)
))

fig.show()


In [ ]:
# fig.write_image(f'{path}/results/Paper/img/feature_citation_graph.pdf')

### Zoom into the centre?


In [31]:
df_feature_edges.head()

,from_type,from_val,to_type,to_val,weight
0,Measure,Regression‐based Measures,RqType,Empirical Estimates and Determinants,913
1,RqType,Empirical Estimates and Determinants,Measure,Regression‐based Measures,702
2,RqType,Intergenerational Wealth Mobility and Inheritance,Measure,Regression‐based Measures,596
3,DataType,Panel/Longitudinal Surveys,Measure,Regression‐based Measures,563
4,Measure,Regression‐based Measures,RqType,Intergenerational Wealth Mobility and Inheritance,458


# By setting a radius around the centroid (0,0)

In [72]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import math

# Load your CSV
df_feature_edges = pd.read_csv(f'{path}/results/weights_properties_citations_noTypeYear.csv')

# Build the graph
G = nx.DiGraph()
for _, row in df_feature_edges.iterrows():
    G.add_edge(row['from_val'], row['to_val'], weight=row['weight'])

# Layout
pos = nx.spring_layout(G, seed=42, k=5 / math.sqrt(G.order()))

# --- CENTRALITY METRICS ---
betweenness = nx.betweenness_centrality(G)
degree = dict(G.degree())

# How many top central nodes to show
top_k = 7  # total = 31

# Top-k nodes by betweenness centrality
top_betweenness_nodes = sorted(betweenness, key=betweenness.get, reverse=True)[:top_k]
# Top-k nodes by degree centrality
top_degree_nodes = sorted(degree, key=degree.get, reverse=True)[:top_k]

# --- GEOMETRIC CENTER ZONE ---
center_radius = 0.1
geometric_center_nodes = [node for node, (x, y) in pos.items() if abs(x) <= center_radius and abs(y) <= center_radius]

# --- CREATE SUBGRAPH FUNCTION ---
def make_subgraph_plot(subgraph_nodes_ordered, centrality_dict, title):
    G_sub = G.subgraph(subgraph_nodes_ordered).copy()
    pos_sub = {n: pos[n] for n in G_sub.nodes()}
    
    edge_weights = [G_sub[u][v]['weight'] for u, v in G_sub.edges()]
    min_w, max_w = min(edge_weights, default=1), max(edge_weights, default=1)
    scaled_widths = [scale_weight(w, min_w, max_w) for w in edge_weights]
    
    # Edge traces
    edge_traces = []
    for (u, v), width in zip(G_sub.edges(), scaled_widths):
        edge_traces.append(go.Scatter(
            x=[pos_sub[u][0], pos_sub[v][0], None],
            y=[pos_sub[u][1], pos_sub[v][1], None],
            mode='lines',
            line=dict(width=width, color='gray'),
            showlegend=False,
        ))

    # Node traces with correct rank labels
    node_traces = []
    for i, node in enumerate(subgraph_nodes_ordered):
        x, y = pos_sub[node]
        rank_label = str(i + 1)  # ranked order (1 = highest centrality, or closest to center)
        node_traces.append(go.Scatter(
            x=[x],
            y=[y],
            mode='markers+text',
            text=[rank_label],
            textposition='middle center',
            textfont=dict(size=12, color='white'),
            marker=dict(
                size=22,
                color=color_labels.get(node, '#cccccc'),
                line=dict(width=2, color='darkblue')
            ),
            hoverinfo='text',
            name=node,
            showlegend=True
        ))

    return go.Figure(data=edge_traces + node_traces, layout=go.Layout(
        title=title,
        showlegend=True,
        hovermode='closest',
        plot_bgcolor='rgba(255,255,255,0)',
        margin=dict(b=20, l=5, r=5, t=40),
        font=dict(size=16, color='black'),
        font_family="Times New Roman",
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False)
    ))



# --- DISPLAY THREE PLOTS ---
# Sort geometric center nodes by distance to origin
geom_sorted = sorted(geometric_center_nodes, key=lambda n: pos[n][0]**2 + pos[n][1]**2)

fig_geom = make_subgraph_plot(geom_sorted, {}, 'Geometric Center Nodes')
fig_betw = make_subgraph_plot(top_betweenness_nodes, betweenness, 'Top Betweenness Centrality Nodes')
fig_deg = make_subgraph_plot(top_degree_nodes, degree, 'Top Degree Centrality Nodes')

fig_geom.show()
fig_betw.show()
fig_deg.show()




In [ ]:
# fig_geom.write_image(f'{path}/results/Paper/img/feature_geom_center_graph.pdf')
# fig_betw.write_image(f'{path}/results/Paper/img/feature_top_betweenness_graph.pdf')
# fig_deg.write_image(f'{path}/results/Paper/img/feature_top_degree_graph.pdf')

# Check the papers with the centroid properties

In [2]:
data = pd.read_csv(f'{path}/data_abstracts/true_mobility_studies_617_forKGs_cleaned.csv')

In [138]:
data.columns

Index(['id', 'title', 'year', 'doi', 'landing_page', 'abstract_inverted_index',
       'language', 'is_oa', 'oa_status', 'oa_link', 'abstract', 'abstract_sm',
       'authors_sm', 'domain', 'sort_gpt_1', 'sort_gpt_2', 'sort_gpt4o_1',
       'sort_gpt4o_2', 'cited_by', 'len_cited_by', 'ref_count',
       'cited_by_count', 'Q1', 'Q1_1', 'Q2', 'Q2_1', 'Q2_2', 'Q3', 'Q4', 'abs',
       'index', 'category_n1', 'measure', 'specified', 'relevant',
       'category_n2', 'measure_1', 'measure_2', 'category_3', 'file', 'id.1',
       'Question1', 'Question2', 'rq_cat', 'RQ', 'data_cat', 'data_type',
       'type', 'category_1', 'category_2', 'author_raw_names',
       'author_raw_affiliations', 'topic_display_names', 'topic_scores',
       'field_display_names', 'subfield_display_names'],
      dtype='object')

In [139]:
data['category_1'].unique()

array(['Absolute Mobility Measures', 'Regression‐based Measures',
       'Transition Matrix / Probability Measures', 'Others',
       'Rank‐based Measures', 'Decomposition / Structural Approaches',
       'Multigenerational Measures', 'Non‐parametric Approaches'],
      dtype=object)

In [140]:
print(data[(data['data_cat'] == "Panel/Longitudinal Surveys") & (data['rq_cat'] == "Empirical Estimates and Determinants") & (data['category_1'] == "Regression‐based Measures")].shape)
data[(data['data_cat'] == "Panel/Longitudinal Surveys") & (data['rq_cat'] == "Empirical Estimates and Determinants") & (data['category_1'] == "Regression‐based Measures")][['id', 'title', 'year', 'data_cat', 'rq_cat', 'category_1', 'category_2']]

(47, 56)


,id,title,year,data_cat,rq_cat,category_1,category_2
9,https://openalex.org/W2411230938,Income Mobility in the United States,2016,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
20,https://openalex.org/W2148814768,Has the Intergenerational Transmission of Econ...,2005,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
24,https://openalex.org/W2802089178,Can cultural consumption increase future earni...,2018,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
46,https://openalex.org/W4238028113,Status Traps,2016,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
72,https://openalex.org/W4308132439,Socioeconomic Mobility of Return Migrants: Evi...,2022,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
73,https://openalex.org/W1020822704,"Poor dad, poor child? An investigation of inte...",2015,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
75,https://openalex.org/W1444704346,Intergenerational Earnings Elasticity Revisite...,2015,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
93,https://openalex.org/W1580804840,Is a College Degree Still the Great Equalizer?...,2011,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
114,https://openalex.org/W1963513809,Patterns of Intergenerational Mobility in Inco...,1992,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN
123,https://openalex.org/W1988669335,THE INTERGENERATIONAL CORRELATION BETWEEN CHIL...,1990,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,NaN


In [141]:
print(data[(data['data_cat'] == "Panel/Longitudinal Surveys") & (data['rq_cat'] == "Intergenerational Wealth Mobility and Inheritance") & (data['category_1'] == "Regression‐based Measures")].shape)
print(data[(data['data_cat'] == "Panel/Longitudinal Surveys") & (data['rq_cat'] == "Intergenerational Wealth Mobility and Inheritance") & (data['category_2'] == "Regression‐based Measures")].shape)

(47, 56)
(6, 56)


In [158]:
data['citation_rank'] = data['cited_by_count'].rank(method='first', ascending=False)
data['citation_rank']

0        4.0
1       22.0
2       34.0
3      132.0
4      217.0
       ...  
612    614.0
613    120.0
614    615.0
615    616.0
616    617.0
Name: citation_rank, Length: 617, dtype: float64

In [180]:
citations_617 = pd.read_csv(f'{path}/results/citations_count_617only.csv')
citations_617 = citations_617.rename(columns={'paper': 'id'})
citations_617 = citations_617.merge(data[['id', 'title', 'data_cat', 'rq_cat', 'category_1', 'category_2', 'citation_rank', 'cited_by_count']], on='id', how='left')
citations_617['citation_rank_617'] = citations_617['citations_count'].rank(method='first', ascending=False)
citations_617.head(10)

,id,citations_count,title,data_cat,rq_cat,category_1,category_2,citation_rank,cited_by_count,citation_rank_617
0,https://openalex.org/W2105584013,169,Where is the land of Opportunity? The Geograph...,Linked Administrative Data,Empirical Estimates and Determinants,Rank‐based Measures,Transition Matrix / Probability Measures,1.0,2341.0,1.0
1,https://openalex.org/W2022256544,150,An Equilibrium Theory of the Distribution of I...,No dataset,Theoretical and Structural Models,Decomposition / Structural Approaches,Decomposition / Structural Approaches,2.0,2152.0,2.0
2,https://openalex.org/W2122216841,131,"Income Inequality, Equality of Opportunity, an...",International Panel Data,Empirical Estimates and Determinants,Regression‐based Measures,NaN,3.0,1529.0,3.0
3,https://openalex.org/W2091916425,112,Fortunate Sons: New Estimates of Intergenerati...,Administrative/Registry Data,Empirical Estimates and Determinants,Regression‐based Measures,NaN,6.0,654.0,4.0
4,https://openalex.org/W3093752946,98,Trends in Intergenerational Income Mobility,Panel/Longitudinal Surveys,Empirical Estimates and Determinants,Regression‐based Measures,Regression‐based Measures,12.0,445.0,5.0
5,https://openalex.org/W2119587652,93,Cross-Country Differences in Intergenerational...,No dataset,Empirical Estimates and Determinants,Regression‐based Measures,NaN,5.0,660.0,6.0
6,https://openalex.org/W2155246844,74,Is the United States Still a Land of Opportuni...,Linked Administrative Data,Empirical Estimates and Determinants,Rank‐based Measures,Rank‐based Measures,7.0,619.0,7.0
7,https://openalex.org/W2082418718,72,Intergenerational Income Mobility Among Daughters,Panel/Longitudinal Surveys,Intergenerational Wealth Mobility and Inheritance,Regression‐based Measures,NaN,15.0,401.0,8.0
8,https://openalex.org/W3121694509,67,The Intergenerational Earnings and Income Mobi...,Linked Administrative Data,Intergenerational Wealth Mobility and Inheritance,Regression‐based Measures,Regression‐based Measures,17.0,299.0,9.0
9,https://openalex.org/W2288802977,58,CROSS‐COUNTRY RANKINGS IN INTERGENERATIONAL MO...,No dataset,Empirical Estimates and Determinants,Regression‐based Measures,NaN,13.0,423.0,10.0


In [ ]:
# citations_617.to_csv(f'{path}/results/citations_count_617only_with_ranks.csv', index=False)

In [3]:
citations_617 = pd.read_csv(f'{path}/results/citations_count_617only_with_ranks.csv')
len(citations_617)

276

In [ ]:
# Using go.Heatmap for more visualization options (instead of go.Histogram2d) ------------------------
import plotly.graph_objects as go

# Create bin edges
variable = 'rq_cat'
citations_617['percentile_bin'] = pd.qcut(
    citations_617['citation_rank_617'],
    q=10,
    labels=[f"P{i*10}-{(i+1)*10}" for i in range(10)],
    duplicates='drop'
)
y_labels = citations_617[f'{variable}']

# Create crosstab
heatmap_data = pd.crosstab(y_labels, citations_617['percentile_bin'], normalize='columns')

fig = go.Figure(
    go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns.astype(str),
        y=heatmap_data.index,
        colorscale='Viridis',
        showscale=True,
        zmin=0,
        # zmax=heatmap_data.values.max(),
        zmax = 0.8,
        hoverongaps=False,
        xgap=2,  # horizontal border space
        ygap=2,  # vertical border space
        colorbar=dict(title='Probability Density')
    )
)

fig.update_layout(
    title=f"Distribution of {variable} across Citation Percentiles",
    xaxis_title="Citation Rank Percentile",
    yaxis_title=f"{variable}",
    # yaxis=dict(showticklabels=False) 
)

fig.show()


### Using Citations_between_617 as weights

In [70]:
import plotly.graph_objects as go
import pandas as pd

# Define the variables you want dropdown control for
variables = ['category_1', 'category_2', 'data_cat', 'rq_cat']

# Bin citation rank
citations_617['percentile_bin'] = pd.qcut(
    citations_617['citation_rank_617'],
    q=10,
    labels=[f"P{i*10}-{(i+1)*10}" for i in range(10)],
    duplicates='drop'
)

# Create color scale
custom_colorscale = [
    [0.0, 'lightgray'],
    [0.000001, 'rgb(68,1,84)'],
    [1.0, 'rgb(253,231,37)']
]

# Store all heatmap traces
traces = []
buttons = []

for i, variable in enumerate(variables):
    pivot_table = (
        citations_617
        .groupby([variable, 'percentile_bin'], observed=True)['citations_count']
        .sum()
        .unstack(fill_value=0)
    )
    pivot_table = pivot_table / pivot_table.values.sum()

    trace = go.Heatmap(
        z=pivot_table.values,
        x=pivot_table.columns.astype(str),
        y=pivot_table.index,
        colorscale=custom_colorscale,
        zmin=0.00001,
        zmax=0.05,  # or pivot_table.values.max() if you want adaptive range
        xgap=2,
        ygap=2,
        visible=(i == 0),  # Only first is visible initially
        colorbar=dict(title="Normalized Citations")
    )
    traces.append(trace)

    # Add dropdown button
    button = dict(
        label=variable,
        method='update',
        args=[
            {'visible': [j == i for j in range(len(variables))]},
            {'title': f'Total Citations by {variable} and Citation Rank Percentile',
             'yaxis': {'title': variable}}
        ]
    )
    buttons.append(button)

# Create the figure with all traces and the dropdown
fig = go.Figure(data=traces)

fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            showactive=True,
            buttons=buttons,
            x=1.05,
            xanchor='left',
            y=1.1,
            yanchor='top'
        )
    ],
    title=f"Total Citations by {variables[0]} and Citation Rank Percentile",
    xaxis_title="Citation Rank Percentile",
    yaxis_title=variables[0]
)

fig.show()



In [ ]:
fig.write_image(f'{path}/results/Paper/img/heatmap_citation_rank_{variable}.pdf')

In [7]:
citations_617['combined'] = citations_617['data_cat'] + ',' + citations_617['rq_cat'] + ',' + citations_617['category_1']
citations_617['combined'] = citations_617['combined'].astype('category')

In [ ]:
citations_617['combined'] = (
    citations_617['data_cat'].astype(str) + ',' +
    citations_617['rq_cat'].astype(str) + ',' +
    citations_617['category_1'].astype(str)
)

combo_counts = (
    citations_617
    .groupby(['percentile_bin', 'combined'], observed=True)
    .size()
    .reset_index(name='count')
)

bin_totals = (
    citations_617
    .groupby('percentile_bin', observed=True)
    .size()
    .reset_index(name='bin_total')
)

combo_counts = combo_counts.merge(bin_totals, on='percentile_bin')
combo_counts['probability'] = combo_counts['count'] / combo_counts['bin_total']

# Select top-N combinations per bin based on true probability
top_n = 5
top_combos = (
    combo_counts
    .sort_values(['percentile_bin', 'probability'], ascending=[True, False])
    .groupby('percentile_bin', observed=True)
    .head(top_n)
)

heatmap_data = pd.pivot_table(
    top_combos,
    values='probability',
    index='combined',
    columns='percentile_bin',
    fill_value=0,
    observed=True
)


fig = go.Figure(
    go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns.astype(str),
        y=heatmap_data.index,
        colorscale='Viridis',
        showscale=True,
        zmin=0,
        # zmax=heatmap_data.values.max(),
        zmax=0.25,
        xgap=2,
        ygap=2,
        colorbar=dict(title='Probability')
    )
)

fig.update_layout(
    title=f"Top {top_n} Feature Combinations per Citation Percentile (True Probability)",
    xaxis_title="Citation Rank Percentile",
    yaxis_title="Feature Combination",
    # yaxis=dict(showticklabels=False)
)

fig.show()


In [31]:
combo_counts[(combo_counts['percentile_bin'] == 'P0-10') & (combo_counts['probability'] > 0)].sort_values('probability', ascending=False)

,percentile_bin,combined,count,bin_total,probability
17,P0-10,"Panel/Longitudinal Surveys,Empirical Estimates...",5,28,0.178571
12,P0-10,"No dataset,Empirical Estimates and Determinant...",4,28,0.142857
6,P0-10,"Linked Administrative Data,Empirical Estimates...",2,28,0.071429
11,P0-10,"National Survey Data,Intergenerational Wealth ...",2,28,0.071429
0,P0-10,"Administrative/Registry Data,Empirical Estimat...",1,28,0.035714
10,P0-10,"Linked Administrative Data,Policy, Institution...",1,28,0.035714
16,P0-10,"Panel/Longitudinal Surveys,Demographic Differe...",1,28,0.035714
15,P0-10,"Others,Empirical Estimates and Determinants,Ot...",1,28,0.035714
14,P0-10,"No dataset,Theoretical and Structural Models,D...",1,28,0.035714
13,P0-10,"No dataset,Intergenerational Wealth Mobility a...",1,28,0.035714


In [ ]:
# fig.write_image(f'{path}/results/Paper/img/heatmap_top_combos_citation_rank.pdf')